In [175]:
import os
import pandas as pd

In [309]:
ai_root = rf'D:\dataset\Nantong_new_out'
case_list = ['P2' + (3-len(str(x)))*'0' + str(x) for x in range(1,203)]
mapping_file = rf'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\performance test\codes\notebook\nantong_mapping2.xlsx'
new_mapping = rf'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\performance test\codes\notebook\nantong_mapping2.xlsx'

In [312]:
def extract_lesion(contents):
    locations, pirads, los = ['','','']
    for content in contents:
        if content.startswith('location (mm)'):
            locations += (content.split(' = ')[1][:-1] + ',')
        if content.startswith('PiradsComb'):
            pirads += (content.split(' = ')[1][:-1] + ',')
        if content.startswith('LoS'):
            los += (content.split(' = ')[1][:-1] + ',')
            
    return [locations, pirads, los]

In [313]:
df = pd.read_excel(mapping_file)
df

,影像号,PID,ISUP,AI_LoS,AI_PIRADs,AI_Lesion_Location,AI_Lesion_PIRADs,AI_Lesion_LoS,GT_PIRADs,GT_Lesion_Location
0,4084704,P2001,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4165219,P2002,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4073197,P2003,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4180474,P2004,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4201437,P2005,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
180,4237696,P2198,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,3951288,P2199,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,3951493,P2200,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,4301105,P2201,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [314]:
folders = os.listdir(ai_root)
for folder in folders:
    if folder in case_list:
        locations, pirads, los = ['','','']
        case_folder = os.path.join(ai_root, folder)
        info_folder = os.path.join(case_folder, os.listdir(case_folder)[0])
        files = os.listdir(info_folder)
        if 'caseLevelLoS.txt' in files:
            with open(os.path.join(info_folder, 'caseLevelLoS.txt'),'r') as f:
                case_los = int(float(f.readlines()[0][:-1]))
        if 'LesionFindings.txt' in files:
            with open(os.path.join(info_folder, 'LesionFindings.txt'), 'r') as f:
                contents = f.readlines()
            locations, pirads, los = extract_lesion(contents)
            try:
                case_pirads = int(float(pirads.split(',')[0]))
            except Exception as e:
                print(locations, pirads, los)
        else:
            case_pirads = 1

                
        df.loc[df['PID']==folder, ['AI_LoS', 'AI_PIRADs', 'AI_Lesion_Location', 'AI_Lesion_PIRADs', 'AI_Lesion_LoS']] = [case_los, case_pirads, locations, pirads, los]

In [315]:
df

,影像号,PID,ISUP,AI_LoS,AI_PIRADs,AI_Lesion_Location,AI_Lesion_PIRADs,AI_Lesion_LoS,GT_PIRADs,GT_Lesion_Location
0,4084704,P2001,0,1.0,1.0,,,,NaN,NaN
1,4165219,P2002,3,99.0,4.0,"-8.86918 53.2745 -12.5108,","4,","99,",NaN,NaN
2,4073197,P2003,4,99.0,5.0,"9.91411 40.3195 -28.2587,","5,","99,",NaN,NaN
3,4180474,P2004,0,52.0,1.0,,,,NaN,NaN
4,4201437,P2005,4,100.0,5.0,"11.0508 56.5872 -9.03066,","5,","100,",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
180,4237696,P2198,0,68.0,3.0,"21.2473 43.7156 -54.3255,","3,","68,",NaN,NaN
181,3951288,P2199,0,55.0,1.0,,,,NaN,NaN
182,3951493,P2200,4,92.0,5.0,"12.6298 62.276 19.1226,","5,","92,",NaN,NaN
183,4301105,P2201,3,100.0,4.0,"-3.34108 48.5516 -11.2424,","4,","100,",NaN,NaN


In [316]:
df.to_excel(new_mapping, index=False)

In [321]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

In [322]:
def roc_pic(fpr, tpr, auc_score, pic_name):
    lw = 2
    plt.figure(figsize=(7,7))
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label=f'ROC curve {auc_score}')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.savefig(pic_name, bbox_inches='tight', dpi=100)

In [325]:
isups = [1,2]

for isup in isups:
    ori_gt, gt, pred = [[],[],[]]
    for input_excel in ['sf1_mapping2.xlsx', 'nantong_mapping2.xlsx']:
        cur_excel = os.path.join(r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\performance test\codes\notebook', input_excel)
        df = pd.read_excel(cur_excel)
        for index, row in df.iterrows():
            try:
                ori_gt.append(int(float(row['ISUP'])))
            except Exception as e:
                print(index)
                continue
            try:
                pred.append(int(float(row['AI_LoS'])))
            except Exception as e:
                print(index)
                ori_gt.pop(-1)
                continue
    gt = [1 if x >= isup else 0 for x in ori_gt]
    fpr, tpr, _ = roc_curve(gt, pred)
    AUC = round(auc(fpr, tpr),4)
    print(AUC)

15
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
0.9282
15
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
0.9374


In [348]:
import SimpleITK as sitk
import numpy as np
import os

lesion_gt = r'D:\dataset\nantong_gt_output'
nantong_mapping = rf'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\performance test\codes\notebook\nantong_mapping2.xlsx'

In [349]:
nantong_df = pd.read_excel(nantong_mapping)

In [350]:
nantong_df.columns

Index(['影像号', 'PID', 'ISUP', 'AI_LoS', 'AI_PIRADs', 'AI_Lesion_Location',
       'AI_Lesion_PIRADs', 'AI_Lesion_LoS', 'GT_PIRADs', 'GT_Lesion_Location'],
      dtype='object')

In [355]:
folders = os.listdir(lesion_gt)
for index, row in nantong_df.iterrows():
    if row.PID in folders:
        mhd_file = os.path.join(lesion_gt, row.PID, 'm0.mhd')
        image = sitk.ReadImage(mhd_file)
        direction = image.GetDirection()
        spacing = image.GetSpacing()
        origin = image.GetOrigin()
        img = sitk.GetArrayFromImage(image)
        gt_pirads = np.max(img)
        nantong_df.loc[nantong_df.PID==row.PID, 'GT_PIRADs'] = gt_pirads


In [356]:
nantong_df.to_excel(nantong_mapping, index=False)

In [361]:
pirads_thresholds = [3,4]

for pirads_threshold in pirads_thresholds:
    ori_gt, gt, pred = [[],[],[]]
    for input_excel in ['nantong_mapping2.xlsx']:
        cur_excel = os.path.join(r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\performance test\codes\notebook', input_excel)
        df = pd.read_excel(cur_excel)
        for index, row in df.iterrows():
            try:
                ori_gt.append(int(float(row['GT_PIRADs'])))
            except Exception as e:
                print(index)
                continue
            try:
                pred.append(int(float(row['AI_LoS'])))
            except Exception as e:
                print(index)
                ori_gt.pop(-1)
                continue
    gt = [1 if x >= pirads_threshold else 0 for x in ori_gt]
    fpr, tpr, _ = roc_curve(gt, pred)
    AUC = round(auc(fpr, tpr),4)
    print(AUC)

47
0.9511
47
0.9429


In [381]:
data_folder = r'D:\dataset\Suzhou_new_out'
for input_excel in ['sf1_mapping2.xlsx']:
    cur_excel = os.path.join(r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\performance test\codes\notebook', input_excel)
    df = pd.read_excel(cur_excel)
    for index, row in df.iterrows():
        cur_case = row.PID
        if cur_case in os.listdir(data_folder):
            case_folder = os.path.join(data_folder, cur_case)
            t2_folder = os.path.join(case_folder, os.listdir(case_folder)[0], 'dicoms','Axial_T2')

            test_dcm = os.path.join(t2_folder, os.listdir(t2_folder)[0])
            ds = pydicom.dcmread(test_dcm)
            age = int(ds.PatientAge[:-1])
            scanner = ds.ManufacturerModelName
            vendor = ds.Manufacturer
            df.loc[df['PID']==cur_case, ['Age','Manufacturer','Scanner']] = age, vendor, scanner
        

In [382]:
df.to_excel(cur_excel)